In [76]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

llm_model = ChatGoogleGenerativeAI(api_key=GOOGLE_API_KEY, model = "gemini-1.5-flash")


In [77]:
result = llm_model.invoke("what is rag in AI in 10 words")
print(result)

content='Retrieval-Augmented Generation: AI using external knowledge for improved responses.' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-6f3a27b4-88bf-48f7-9505-c9501592fd53-0' usage_metadata={'input_tokens': 11, 'output_tokens': 15, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}}


# Load PDF

In [78]:
from langchain_community.document_loaders import PyPDFLoader


loader = PyPDFLoader("pdfs\case study.pdf")
documents = loader.load_and_split()
documents

[Document(metadata={'source': 'pdfs\\case study.pdf', 'page': 0}, page_content='Case Studies \n“I’ve learned that people will forget what you’ve said, people will forget what \nyou did, but people will never forget how you made them feel.”  Maya Angelou \nWhere case studies can be used: \n\uf0b7 Websites (ideally front page or one click away) \n\uf0b7 Fundraising Applications \n\uf0b7 Annual Reports \n\uf0b7 Press releases \n\uf0b7 Newsletters \n\uf0b7 Social media \n\uf0b7 Fundraising events (eg with client in person) \n\uf0b7 Conferences \n\uf0b7 Policy briefings \n\uf0b7 Volunteer recruitment \n\uf0b7 Radio Appeals/Podcasts \n \nTips for obtaining case studies \n \n \nChoose a case that either is typical of your clinic’s work, or that demonstrates a current policy issue.  \n \n \nWhen you ask a client for permission to use their story, explain to them the purposes for which that \nthe case study will be used and how this will help the clinic, and other people like them, in the \nfut

# pages of pdf

In [79]:
# Extract the text content from the Document objects
pages = [doc.page_content for doc in documents]
pages


['Case Studies \n“I’ve learned that people will forget what you’ve said, people will forget what \nyou did, but people will never forget how you made them feel.”  Maya Angelou \nWhere case studies can be used: \n\uf0b7 Websites (ideally front page or one click away) \n\uf0b7 Fundraising Applications \n\uf0b7 Annual Reports \n\uf0b7 Press releases \n\uf0b7 Newsletters \n\uf0b7 Social media \n\uf0b7 Fundraising events (eg with client in person) \n\uf0b7 Conferences \n\uf0b7 Policy briefings \n\uf0b7 Volunteer recruitment \n\uf0b7 Radio Appeals/Podcasts \n \nTips for obtaining case studies \n \n \nChoose a case that either is typical of your clinic’s work, or that demonstrates a current policy issue.  \n \n \nWhen you ask a client for permission to use their story, explain to them the purposes for which that \nthe case study will be used and how this will help the clinic, and other people like them, in the \nfuture.  For instance, this will help the clinic to raise money; this will help u

# text splitter

In [80]:
from langchain_text_splitters import RecursiveCharacterTextSplitter , CharacterTextSplitter

text_split = CharacterTextSplitter(chunk_size=100, chunk_overlap=50)
chunks = text_split.create_documents(pages)

# Display results
print(f"Number of chunks: {len(chunks)}")
print(f"Type of first chunk: {type(chunks[0])}")

Number of chunks: 4
Type of first chunk: <class 'langchain_core.documents.base.Document'>


# Embeddings

In [81]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embedding_model = GoogleGenerativeAIEmbeddings(google_api_key= GOOGLE_API_KEY,  model="models/text-embedding-004",  chunk_size=100,task_type="retrieval_document")


embedding_model


GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x00000101C3C55760>, model='models/text-embedding-004', task_type='retrieval_document', google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

# faiss vector-db

In [82]:
from langchain.vectorstores import FAISS
vector_store = FAISS.from_texts(pages, embedding=embedding_model)
vector_store.save_local("faiss_index")

In [83]:
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

print(type(retriever))

<class 'langchain_core.vectorstores.base.VectorStoreRetriever'>


# Prompt template

In [84]:
from langchain_core.messages import SystemMessage
from langchain_core.prompts import ChatMessagePromptTemplate, ChatPromptTemplate , HumanMessagePromptTemplate


In [85]:
chat_template = ChatPromptTemplate.from_messages([
    # System Message Prompt Template
    SystemMessage(content="""You are an expert Legal Research Assistant specializing in case law analysis and legal drafting.
                 Given case law context and a question from the user, you should:
                 1. Analyze the legal principles and precedents from the provided cases
                 2. Identify relevant holdings and ratio decidend
                 3. Provide well-reasoned legal analysis with proper citations
                 4. When drafting, follow proper legal writing conventions and structure
                 
                 Format your responses with:
                 - Clear headings for different sections
                 - Proper case citations
                 - Specific references to relevant passages from the context
                 - Clear distinction between holdings and obiter dicta
                 - Practical applications or implications when relevant"""),
    
    # Human Message Prompt Template
    HumanMessagePromptTemplate.from_template("""Analyze the following legal materials and answer the question provided.
    
    Case Law Context: {context}
    
    Legal Query: {question}
    
    Analysis: """)
])

In [86]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [87]:
from langchain_core.runnables import RunnablePassthrough
from datetime import datetime
def format_legal_docs(docs):
    """Format legal documents with proper structure and citations"""
    formatted_docs = []
    for i, doc in enumerate(docs, 1):
        # Extract metadata if available
        citation = doc.metadata.get('citation', 'No citation available')
        date = doc.metadata.get('date', 'No date available')
        court = doc.metadata.get('court', 'Court information not available')
        
        # Format each document with legal context
        formatted_text = f"""
Case Document {i}
Citation: {citation}
Court: {court}
Date: {date}
---
{doc.page_content}
---"""
        formatted_docs.append(formatted_text)
    
    return "\n\n".join(formatted_docs)

# Define the RAG chain with legal processing
rag_chain = (
    {
        "context": retriever | format_legal_docs,
        "question": RunnablePassthrough(),
        "metadata": lambda x: {
            "query_type": "legal_analysis",
            "timestamp": datetime.now().isoformat()
        }
    }
    | chat_template
    | llm_model
    | output_parser
)

In [90]:
response = rag_chain.invoke("As Lawyer create drafting of this case:Robert’s story")
print(response)

## Case Study: Robert's Story

**I. Introduction**

This case study details the experience of Robert, a 61-year-old man, in navigating the complexities of the Employment Support Allowance (ESA) system following a debilitating workplace accident.  His case highlights the critical role of legal aid in ensuring access to justice for vulnerable individuals facing benefit appeals.  Due to legal aid cuts stemming from the Legal Aid, Sentencing and Punishment of Offenders Act 2012, Robert's successful appeal stands as a testament to the vital assistance provided by organizations like LawWorks.

**II. Factual Background**

Robert, a former decorator, suffered a severe leg injury 28 years prior, resulting in a shortened leg and subsequent health complications. Despite his injuries, he worked in a physically demanding job at a print works for many years.  However, his health deteriorated, forcing him to cease working five years ago.  He subsequently applied for and received ESA.

Last year, Robe